### Pandas를 활용하여 영화 평점 데이터 분석하기

- MovieLens 사용자들을 대상으로 수집한 영화 평점 데이터를 분석하기
- 데이터 출처 : https://grouplens.org/datasets/movielens/
- 데이터 내용 : 영화 평점, 장르, 개봉연도, 사용자 나이, 우편번호, 성별, 직업 등이 포함

- 실습 참조 : 
    - 파이썬 라이브러리를 활용한 데이터 분석
    - https://sandipanweb.wordpress.com/2017/12/16/data-science-with-python-exploratory-analysis-with-movie-ratings-and-fraud-detection-with-credit-card-transactions/

### 데이터 읽기

- 해당 데이터셋은 평점, 사용자 정보, 영화 정보 테이블로 구성되어있어서 각 파일을 따로 읽어와야 한다.

In [9]:
import pandas as pd

In [13]:
# 나이와 직업은 그룹을 가리키는 코드 번호이다
users = pd.read_table('./data/ml-1m/users.dat',\
                      sep='::', \
                      header=None, \
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'])

users.head()

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [35]:
movies = pd.read_table('./data/ml-1m/movies.dat',\
                      sep='::',\
                      header=None,\
                      names=['movie_id', 'title', 'genres'])
movies.head()

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
ratings = pd.read_table('./data/ml-1m/ratings.dat',\
                       sep='::',\
                       header=None,\
                       names=['user_id', 'movie_id', 'rating', 'timestamp'])
ratings.head()

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 데이터 병합하기

- 모든 데이터를 하나의 테이블로 병합해서 처리하는 것이 더 편리하기 때문에 먼저 ratings와 users를 병합하고 이후에 movies와 병합한다.
- merge는 병합하는 두 테이블에서 중복되는 열의 이름을 키로 사용한다.

In [24]:
# ratings + users
data = pd.merge(ratings, users)
# + moview
data = pd.merge(data, movies)

data.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


### 영화 장르에 따른 평점의 변화 분석하기

#### 데이터 전처리

In [30]:
# 장르 갯수 확인하기

# 하나의 영화는 하나의 장르가 아니라 여러 장르에 속할 수 있다
movies.genres.unique()[:10]

array(["Animation|Children's|Comedy", "Adventure|Children's|Fantasy",
       'Comedy|Romance', 'Comedy|Drama', 'Comedy',
       'Action|Crime|Thriller', "Adventure|Children's", 'Action',
       'Action|Adventure|Thriller', 'Comedy|Drama|Romance'], dtype=object)

In [51]:
# 각 영화 레코드를 장르에 따라 각각 추가하는 방식으로 전처리
# stack()는 데이터를 재구조화 하는 메소드
movies_ = pd.DataFrame(movies.genres.str.split('|').tolist(), \
                 index=movies.movie_id).stack()
movies_.head()

movie_id   
1         0     Animation
          1    Children's
          2        Comedy
2         0     Adventure
          1    Children's
dtype: object

unstacked

|index |col_1 | col_2|
|----|----|----|
|1 | 10 | 30 |
|2 | 20 | 40 |

stacked

|index|index| |
|---|-----|---|
|1 | col_1| 10|
|1 | col_1| 20|
|2 | col_2| 30|
|2 | col_2| 40|

In [54]:
movies_ = movies_.reset_index()[[0, 'movie_id']]
movies_.columns = ['genres', 'movie_id']
movies_.head()

,genres,movie_id
0,Animation,1
1,Children's,1
2,Comedy,1
3,Adventure,2
4,Children's,2


In [55]:
# 데이터가 제대로 반영되었는가 확인을 위해 비교 데이터 찾기
movies.loc[movies.movie_id==1]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
